In [ ]:
import transformers
from transformers import LongformerTokenizer, LongformerModel, BertTokenizer, AdamW, get_linear_schedule_with_warmup
import torch

import numpy as np
import pandas as pd
import re
from tqdm.auto import tqdm

import seaborn as sns
from pylab import rcParams
import matplotlib.pyplot as plt
from matplotlib import rc
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold
from sklearn.metrics import multilabel_confusion_matrix, classification_report
from collections import defaultdict
from textwrap import wrap

from torch import nn, optim
from torch.utils.data import Dataset, DataLoader

import pytorch_lightning as pl
from torchmetrics.functional import accuracy, auroc
from pytorch_lightning.callbacks import ModelCheckpoint, EarlyStopping
from pytorch_lightning.loggers import TensorBoardLogger

#matplotlib setting/format specifications

%matplotlib inline
%config InlineBackend.figure_format='retina'

sns.set(style='whitegrid', palette='muted', font_scale=1.2)

HAPPY_COLORS_PALETTE = ["#01BEFE", "#FFDD00", "#FF7D00", "#FF006D", "#ADFF02", "#8F00FF"]

sns.set_palette(sns.color_palette(HAPPY_COLORS_PALETTE))

#Controls size of figures we create
rcParams['figure.figsize'] = 8, 6

#set random seed

RANDOM_SEED = 42
np.random.seed(RANDOM_SEED)
torch.manual_seed(RANDOM_SEED)

/usr/local/lib/python3.7/dist-packages/torchvision/io/image.py:13: UserWarning: Failed to load image Python extension: libtorch_cuda_cu.so: cannot open shared object file: No such file or directory
  warn(f"Failed to load image Python extension: {e}")


In [ ]:
sw_notes=pd.read_csv('Final_SWmerged.csv')

In [ ]:
def clean_text(df):
  text = str(df["TEXT"])
  clean = re.sub(r"\n","",text)
  cleaner = re.sub(r"  ","",clean)

  return cleaner

In [ ]:
sw_notes["TEXT_CLEAN"] = sw_notes.apply(clean_text, axis=1)

In [ ]:
LABEL_COLUMNS = sw_notes.columns.tolist()[11:15]

## Dataset Class

In [ ]:
#Define tokenizer
tokenizer = LongformerTokenizer.from_pretrained("allenai/longformer-base-4096",max_length=1000)

Downloading:   0%|          | 0.00/899k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/456k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/694 [00:00<?, ?B/s]

In [ ]:
class SWNotesDataset(Dataset):

  def __init__(self, data: pd.DataFrame, tokenizer:BertTokenizer, max_len=512):
    self.data = data
    self.tokenizer = tokenizer
    self.max_len = max_len

  def __len__(self):
    return(len(self.data))

  def __getitem__(self,index: int):
    data_row = self.data.iloc[index]
    note = data_row.TEXT_CLEAN
    labels = data_row[LABEL_COLUMNS]

    encoding=self.tokenizer.encode_plus(
        note,
        add_special_tokens=True,
        truncation=True,
        max_length=self.max_len,
        return_token_type_ids=False,
        padding= 'max_length',
        return_attention_mask=True,
        return_tensors='pt',
    )

    return {
        'SW_note': note,
        'input_ids': encoding['input_ids'].flatten(),
        'attention_mask': encoding['attention_mask'].flatten(),
        'labels': torch.FloatTensor(labels)
    }

In [ ]:
#Split data into 5 folds

kf = KFold(n_splits=5, shuffle=True,random_state=RANDOM_SEED)
folds = kf.split(sw_notes)

trn_idx1,tst_idx1 = next(folds)
trn_idx2,tst_idx2 = next(folds)
trn_idx3,tst_idx3 = next(folds)
trn_idx4,tst_idx4 = next(folds)
trn_idx5,tst_idx5 = next(folds)

In [ ]:
df_train1 = sw_notes.iloc[trn_idx1]
df_train2 = sw_notes.iloc[trn_idx2]
df_train3 = sw_notes.iloc[trn_idx3]
df_train4 = sw_notes.iloc[trn_idx4]
df_train5 = sw_notes.iloc[trn_idx5]

df_test1 = sw_notes.iloc[tst_idx1]
df_test2 = sw_notes.iloc[tst_idx2]
df_test3 = sw_notes.iloc[tst_idx3]
df_test4 = sw_notes.iloc[tst_idx4]
df_test5 = sw_notes.iloc[tst_idx5]

In [ ]:
class SWNotesDataModule(pl.LightningDataModule):
  def __init__(self, df_train, df_test, tokenizer, batch_size=8, max_len=512):
    super().__init__()
    self.batch_size = batch_size
    self.df_train = df_train
    self.df_test = df_test
    self.tokenizer = tokenizer
    self.max_len = max_len

  def setup(self, stage=None):

    self.train_dataset = SWNotesDataset(
      self.df_train,
      self.tokenizer,
      self.max_len
    )

    self.test_dataset = SWNotesDataset(
      self.df_test,
      self.tokenizer,
      self.max_len
    )


  def train_dataloader(self):
    return DataLoader(
      self.train_dataset,
      batch_size=self.batch_size,
      shuffle=True,
      num_workers=2
    )

  def val_dataloader(self):
    return DataLoader(
      self.test_dataset,
      batch_size=self.batch_size,
      num_workers=2
    )

  def test_dataloader(self):
    return DataLoader(
      self.test_dataset,
      batch_size=self.batch_size,
      num_workers=2
    )



In [ ]:
N_EPOCHS = 10
BATCH_SIZE = 4
MAX_LEN = 1000

In [ ]:
#Fold changes each run
data_module = SWNotesDataModule(
  df_train5,
  df_test5,
  tokenizer,
  batch_size=BATCH_SIZE,
  max_len=MAX_LEN
)

## Building Classification Model

In [ ]:
#Pytorch lightning method

class SWRoleClassifier(pl.LightningModule):

  def __init__(self, n_classes: int, n_training_steps=None, n_warmup_steps=None):
    super().__init__()
    self.bert = LongformerModel.from_pretrained("allenai/longformer-base-4096", return_dict=True)
    self.classifier = nn.Linear(self.bert.config.hidden_size, n_classes)
    self.n_training_steps = n_training_steps
    self.n_warmup_steps = n_warmup_steps
    self.criterion = nn.BCELoss()

  def forward(self, input_ids, attention_mask, labels=None):
    output = self.bert(input_ids, attention_mask=attention_mask)
    output = self.classifier(output.pooler_output)
    output = torch.sigmoid(output)
    loss = 0
    if labels is not None:
        loss = self.criterion(output, labels)
    return loss, output

  def training_step(self, batch, batch_idx):
    input_ids = batch["input_ids"]
    attention_mask = batch["attention_mask"]
    labels = batch["labels"]
    loss, outputs = self(input_ids, attention_mask, labels)
    self.log("train_loss", loss, prog_bar=True, logger=True)
    return {"loss": loss, "predictions": outputs, "labels": labels}

  def validation_step(self, batch, batch_idx):
    input_ids = batch["input_ids"]
    attention_mask = batch["attention_mask"]
    labels = batch["labels"]
    loss, outputs = self(input_ids, attention_mask, labels)
    self.log("val_loss", loss, prog_bar=True, logger=True)
    return loss

  def test_step(self, batch, batch_idx):
    input_ids = batch["input_ids"]
    attention_mask = batch["attention_mask"]
    labels = batch["labels"]
    loss, outputs = self(input_ids, attention_mask, labels)
    self.log("test_loss", loss, prog_bar=True, logger=True)
    return loss

  def training_epoch_end(self, outputs):
    labels = []
    predictions = []
    for output in outputs:
      for out_labels in output["labels"].detach().cpu():
        labels.append(out_labels)
      for out_predictions in output["predictions"].detach().cpu():
        predictions.append(out_predictions)

    labels = torch.stack(labels).int()
    predictions = torch.stack(predictions)


  def configure_optimizers(self):
    optimizer = AdamW(self.parameters(), lr=2e-5)
    scheduler = get_linear_schedule_with_warmup(
      optimizer,
      num_warmup_steps=self.n_warmup_steps,
      num_training_steps=self.n_training_steps
    )

    return dict(
      optimizer=optimizer,
      lr_scheduler=dict(
        scheduler=scheduler,
        interval='step'
      )
    )

In [ ]:
#Calculate warm-up and total steps
steps_per_epoch=len(df_train1) // BATCH_SIZE
total_training_steps = steps_per_epoch * N_EPOCHS

warmup_steps = total_training_steps // 5
warmup_steps, total_training_steps

(502, 2510)

In [ ]:
model = SWRoleClassifier(
  n_classes=len(LABEL_COLUMNS),
  n_warmup_steps=warmup_steps,
  n_training_steps=total_training_steps
)

Downloading:   0%|          | 0.00/597M [00:00<?, ?B/s]

Some weights of the model checkpoint at allenai/longformer-base-4096 were not used when initializing LongformerModel: ['lm_head.layer_norm.weight', 'lm_head.dense.weight', 'lm_head.dense.bias', 'lm_head.bias', 'lm_head.layer_norm.bias', 'lm_head.decoder.weight']
- This IS expected if you are initializing LongformerModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing LongformerModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


## Training

In [ ]:
logger = TensorBoardLogger("lightning_logs", name="SWroles2")

early_stopping_callback = EarlyStopping(monitor='val_loss', patience=3)

In [ ]:
trainer = pl.Trainer(
  logger=logger,
  log_every_n_steps=40,
  callbacks=[early_stopping_callback],
  max_epochs=N_EPOCHS,
  gpus=1
)

/usr/local/lib/python3.7/dist-packages/pytorch_lightning/trainer/connectors/accelerator_connector.py:447: LightningDeprecationWarning: Setting `Trainer(gpus=1)` is deprecated in v1.7 and will be removed in v2.0. Please use `Trainer(accelerator='gpu', devices=1)` instead.
  f"Setting `Trainer(gpus={gpus!r})` is deprecated in v1.7 and will be removed"
INFO:pytorch_lightning.utilities.rank_zero:GPU available: True (cuda), used: True
INFO:pytorch_lightning.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO:pytorch_lightning.utilities.rank_zero:IPU available: False, using: 0 IPUs
INFO:pytorch_lightning.utilities.rank_zero:HPU available: False, using: 0 HPUs


In [ ]:
#Training step
trainer.fit(model, data_module)

INFO:pytorch_lightning.accelerators.cuda:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
/usr/local/lib/python3.7/dist-packages/transformers/optimization.py:310: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,
INFO:pytorch_lightning.callbacks.model_summary:
  | Name       | Type            | Params
-----------------------------------------------
0 | bert       | LongformerModel | 148 M 
1 | classifier | Linear          | 3.1 K 
2 | criterion  | BCELoss         | 0     
-----------------------------------------------
148 M     Trainable params
0         Non-trainable params
148 M     Total params
594.650   Total estimated model params size (MB)


Sanity Checking: 0it [00:00, ?it/s]

Training: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

...

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

INFO:pytorch_lightning.utilities.rank_zero:`Trainer.fit` stopped: `max_epochs=10` reached.


## Evaluation

In [ ]:
#Set model to eval mode

trained_model = SWRoleClassifier.load_from_checkpoint(
  trainer.checkpoint_callback.best_model_path,
  n_classes=len(LABEL_COLUMNS)
)

trained_model.eval()
trained_model.freeze()

trained_model = trained_model.to(device)

Some weights of the model checkpoint at allenai/longformer-base-4096 were not used when initializing LongformerModel: ['lm_head.layer_norm.weight', 'lm_head.dense.weight', 'lm_head.dense.bias', 'lm_head.bias', 'lm_head.layer_norm.bias', 'lm_head.decoder.weight']
- This IS expected if you are initializing LongformerModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing LongformerModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [ ]:
#Change fold each run
test_data = SWNotesDataset(
    data = df_test5,
    tokenizer=tokenizer,
    max_len=MAX_LEN
  )

predictions = []
labels = []

In [ ]:
for item in tqdm(test_data):
  _, prediction = trained_model(
    item["input_ids"].unsqueeze(dim=0).to(device),
    item["attention_mask"].unsqueeze(dim=0).to(device)
  )
  predictions.append(prediction.flatten())
  labels.append(item["labels"].int())

  0%|          | 0/251 [00:00<?, ?it/s]

In [ ]:
predictions = torch.stack(predictions).detach().cpu()
labels = torch.stack(labels).detach().cpu()

In [ ]:
#Accuracy
accuracy(predictions, labels, threshold=0.5)

tensor(0.8954)

In [ ]:
#Fold 1
y_pred = predictions.numpy()
y_true = labels.numpy()

upper, lower = 1, 0

y_pred = np.where(y_pred > 0.5, upper, lower)

print(classification_report(
  y_true,
  y_pred,
  target_names=LABEL_COLUMNS,
  zero_division=0
))

                            precision    recall  f1-score   support

          Involved_Support       0.91      0.94      0.92        94
Communication_Facilitation       0.62      0.73      0.67        59
                Counseling       0.61      0.66      0.63        35
      Practical_Assistance       0.75      0.70      0.73        57

                 micro avg       0.75      0.79      0.77       245
                 macro avg       0.72      0.76      0.74       245
              weighted avg       0.76      0.79      0.77       245
               samples avg       0.49      0.50      0.48       245



In [ ]:
#Fold 2
y_pred = predictions.numpy()
y_true = labels.numpy()

upper, lower = 1, 0

y_pred = np.where(y_pred > 0.5, upper, lower)

print(classification_report(
  y_true,
  y_pred,
  target_names=LABEL_COLUMNS,
  zero_division=0
))



                            precision    recall  f1-score   support

          Involved_Support       0.89      0.92      0.90        86
Communication_Facilitation       0.68      0.61      0.64        56
                Counseling       0.50      0.55      0.52        29
      Practical_Assistance       0.78      0.61      0.69        57

                 micro avg       0.76      0.72      0.74       228
                 macro avg       0.71      0.67      0.69       228
              weighted avg       0.76      0.72      0.74       228
               samples avg       0.45      0.44      0.43       228



In [ ]:
#Fold 3
y_pred = predictions.numpy()
y_true = labels.numpy()

upper, lower = 1, 0

y_pred = np.where(y_pred > 0.5, upper, lower)

print(classification_report(
  y_true,
  y_pred,
  target_names=LABEL_COLUMNS,
  zero_division=0
))


                            precision    recall  f1-score   support

          Involved_Support       0.82      0.90      0.86        90
Communication_Facilitation       0.59      0.54      0.56        68
                Counseling       0.85      0.49      0.62        35
      Practical_Assistance       0.74      0.66      0.70        59

                 micro avg       0.74      0.69      0.71       252
                 macro avg       0.75      0.65      0.68       252
              weighted avg       0.74      0.69      0.71       252
               samples avg       0.47      0.46      0.45       252



In [ ]:
#Fold 4
y_pred = predictions.numpy()
y_true = labels.numpy()

upper, lower = 1, 0

y_pred = np.where(y_pred > 0.5, upper, lower)

print(classification_report(
  y_true,
  y_pred,
  target_names=LABEL_COLUMNS,
  zero_division=0
))


                            precision    recall  f1-score   support

          Involved_Support       0.89      0.90      0.90        90
Communication_Facilitation       0.56      0.69      0.62        51
                Counseling       0.64      0.42      0.51        38
      Practical_Assistance       0.77      0.64      0.70        58

                 micro avg       0.75      0.71      0.73       237
                 macro avg       0.72      0.66      0.68       237
              weighted avg       0.75      0.71      0.73       237
               samples avg       0.47      0.44      0.44       237



In [ ]:
#Fold 5
y_pred = predictions.numpy()
y_true = labels.numpy()

upper, lower = 1, 0

y_pred = np.where(y_pred > 0.5, upper, lower)

print(classification_report(
  y_true,
  y_pred,
  target_names=LABEL_COLUMNS,
  zero_division=0
))


                            precision    recall  f1-score   support

          Involved_Support       0.89      0.93      0.91        76
Communication_Facilitation       0.55      0.76      0.64        55
                Counseling       0.56      0.41      0.47        22
      Practical_Assistance       0.72      0.74      0.73        42

                 micro avg       0.71      0.78      0.74       195
                 macro avg       0.68      0.71      0.69       195
              weighted avg       0.72      0.78      0.74       195
               samples avg       0.38      0.39      0.38       195

